<a href="https://colab.research.google.com/github/SergeyShchus/Open-Data/blob/main/Notebook/%D0%93%D0%B5%D0%BE%D0%B0%D0%BD%D0%B0%D0%BB%D1%96%D0%B7_%D1%83_%D0%BC%D1%96%D1%81%D1%82%D1%96_%D0%B7_%D0%B2%D0%B8%D0%BA%D0%BE%D1%80%D0%B8%D1%81%D1%82%D0%B0%D0%BD%D0%BD%D1%8F%D0%BC_%D0%B3%D0%B5%D0%BA%D1%81%D0%B0%D0%B3%D0%BE%D0%BD%D1%96%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#установка бібліотек

!apt-get install software-properties-common python-software-properties > /dev/null
!add-apt-repository ppa:ubuntugis/ppa -y > /dev/null
!apt-get update > /dev/null
!apt-get install -y --fix-missing python-gdal gdal-bin libgdal-dev > /dev/null
!pip2 install OpticalRS > /dev/null

! pip install geopandas

E: Package 'python-software-properties' has no installation candidate
E: Package 'python-gdal' has no installation candidate
/bin/bash: pip2: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install h3
!pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import geopandas as gpd

import pandas as pd

import numpy as np

import json

import h3

import folium

import osmnx as ox

from shapely import wkt

from folium.plugins import HeatMap

from shapely.geometry import Polygon

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
#створення ячейки гексагон


def visualize_hexagons(hexagons, color="red", folium_map=None):

 

    polylines = []

    lat = []

    lng = []

    for hex in hexagons:

        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)

        outlines = [loop for polygon in polygons for loop in polygon]

        polyline = [outline + [outline[0]] for outline in outlines][0]

        lat.extend(map(lambda v:v[0],polyline))

        lng.extend(map(lambda v:v[1],polyline))

        polylines.append(polyline)

   

    if folium_map is None:

        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=20, tiles='cartodbpositron')

    else:

        m = folium_map

       

    for polyline in polylines:

        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)

        m.add_child(my_PolyLine)

    return m

 

h3_address = h3.geo_to_h3(50.434403, 30.496414,  9) # 9 - индекс, определяющий размер гексагона                                                                                                    

visualize_hexagons([h3_address])

In [10]:
#створення області інтересів на основі даних OSM


def visualize_polygons(geometry):

   

    lats, lons = get_lat_lon(geometry)

   

    m = folium.Map(location=[sum(lats)/len(lats), sum(lons)/len(lons)], zoom_start=13, tiles='cartodbpositron')

   

    overlay = gpd.GeoSeries(geometry).to_json()

    folium.GeoJson(overlay, name = 'boundary').add_to(m)

   

    return m

 

# выводим центроиды полигонов

def get_lat_lon(geometry):

       

    lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)

    lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)

    return lat, lon

 

# выгрузим границы Краснодара из OSM

cities = ['Київ']

polygon_krd = ox.geometries_from_place(cities, {'boundary':'administrative'}).reset_index()

polygon_krd = polygon_krd[(polygon_krd['name'] == 'Київ')]

# посмотрим что получилось

visualize_polygons(polygon_krd['geometry'])

In [11]:
#заповнення області інтересів  попередньо згенерованими ячейками для аналізу


def create_hexagons(geoJson):

   

    polyline = geoJson['coordinates'][0]

 

    polyline.append(polyline[0])

    lat = [p[0] for p in polyline]

    lng = [p[1] for p in polyline]

    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')

    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color="green")

    m.add_child(my_PolyLine)

 

    hexagons = list(h3.polyfill(geoJson, 8))

    polylines = []

    lat = []

    lng = []

    for hex in hexagons:

        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)

        # flatten polygons into loops.

        outlines = [loop for polygon in polygons for loop in polygon]

        polyline = [outline + [outline[0]] for outline in outlines][0]

        lat.extend(map(lambda v:v[0],polyline))

        lng.extend(map(lambda v:v[1],polyline))

        polylines.append(polyline)

    for polyline in polylines:

        my_PolyLine=folium.PolyLine(locations=polyline,weight=3,color='red')

        m.add_child(my_PolyLine)

       

    polylines_x = []

    for j in range(len(polylines)):

        a = np.column_stack((np.array(polylines[j])[:,1],np.array(polylines[j])[:,0])).tolist()

        polylines_x.append([(a[i][0], a[i][1]) for i in range(len(a))])

       

    polygons_hex = pd.Series(polylines_x).apply(lambda x: Polygon(x))

       

    return m, polygons_hex, polylines

# polygon_hex , polylines - геометрии гексагонов в разных форматах

 

# сгенерим гексагоны внутри полигона г. Краснодар

geoJson = json.loads(gpd.GeoSeries(polygon_krd['geometry']).to_json())

geoJson = geoJson['features'][0]['geometry']

geoJson = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson['coordinates'][0])[:, 1],

                                                      np.array(geoJson['coordinates'][0])[:, 0])).tolist()]}

 

m, polygons, polylines = create_hexagons(geoJson)

m

 

In [13]:
#викачування з OSM геометрій об'єктів шо цікавлять
 

def osm_query(tag, city):

    gdf = ox.geometries_from_place(city, tag).reset_index()

    gdf['city'] = np.full(len(gdf), city.split(',')[0])

    gdf['object'] = np.full(len(gdf), list(tag.keys())[0])

    gdf['type'] = np.full(len(gdf), tag[list(tag.keys())[0]])

    gdf = gdf[['city', 'object', 'type', 'geometry']]

    print(gdf.shape)

    return gdf

 

 # Выгрузим интересующие нас категории объектов

tags = [

        {'building' : 'apartments'}, {'building' : 'detached'},

        {'building' : 'dormitory'}, {'building' : 'hotel'},

        {'building' : 'house'}, {'building' : 'semidetached_house'},

        {'building' : 'terrace'},  {'building' : 'commercial'},

        {'building' : 'office'},  {'building' : 'terrace'}, 

        {'building' : 'terrace'}, {'building':'retail'},

        {'building':'train_station'},

        {'highway' : 'bus_stop'}, {'footway':'crossing'},

        {'amenity':'cafe'}, {'amenity':'fast_food'},

        {'amenity':'restaurant'}, {'amenity':'college'},

        {'amenity':'language_school'},  {'amenity':'school'}, 

        {'amenity':'university'},  {'amenity':'atm'}, 

        {'amenity':'bank'},  {'amenity':'clinic'}, 

        {'amenity':'hospital'},  {'amenity':'pharmacy'}, 

        {'amenity':'theatre'},  {'amenity':'townhall'}, 

        {'amenity':'bench'},

       ]

cities = ['Київ, Україна']

 

gdfs = []

for city in cities:

    for tag in tags:

        gdfs.append(osm_query(tag, city))

       

# посмотрим что получилось

data_poi = pd.concat(gdfs)

data_poi.groupby(['city','object','type'], as_index = False).agg({'geometry':'count'})

 

# добавим координаты/центроиды

lat, lon = get_lat_lon(data_poi['geometry'])

data_poi['lat'] = lat

data_poi['lon'] = lon

(9446, 4)
(405, 4)
(171, 4)
(27, 4)
(4827, 4)
(3, 4)
(9, 4)
(583, 4)
(553, 4)
(9, 4)
(9, 4)
(975, 4)
(9, 4)
(2652, 4)
(2755, 4)
(1668, 4)
(541, 4)
(1043, 4)
(63, 4)
(31, 4)
(525, 4)
(122, 4)
(1226, 4)
(1180, 4)
(353, 4)
(132, 4)
(1232, 4)
(54, 4)
(17, 4)
(4477, 4)


In [14]:
# поєднання об'єктів з ячейками (intersect)

# sjoin - spatial join

 

gdf_1 = gpd.GeoDataFrame(data_poi, geometry=gpd.points_from_xy(data_poi.lon, data_poi.lat))

 

gdf_2 = pd.DataFrame(polygons, columns = ['geometry'])

gdf_2['polylines'] = polylines

gdf_2['geometry'] = gdf_2['geometry'].astype(str)

geometry_uniq = pd.DataFrame(gdf_2['geometry'].drop_duplicates())

geometry_uniq['id'] = np.arange(len(geometry_uniq)).astype(str)

gdf_2 = gdf_2.merge(geometry_uniq, on = 'geometry')

gdf_2['geometry'] = gdf_2['geometry'].apply(wkt.loads)

gdf_2 = gpd.GeoDataFrame(gdf_2, geometry='geometry')

 

itog_table = gpd.sjoin(gdf_2, gdf_1, how='left', op='intersects')

itog_table = itog_table.dropna()

itog_table.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3249: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-14-11ddd3a9c242>:29: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  itog_table = gpd.sjoin(gdf_2, gdf_1, how='left', op='intersects')


,geometry,polylines,id,index_right,city,object,type,lat,lon
1,"POLYGON ((30.50671 50.41652, 30.49918 50.41632...","[(50.41652102209144, 30.506706589076238), (50....",1,532.0,Київ,amenity,cafe,50.410308,30.508032
1,"POLYGON ((30.50671 50.41652, 30.49918 50.41632...","[(50.41652102209144, 30.506706589076238), (50....",1,826.0,Київ,highway,bus_stop,50.410846,30.507722
1,"POLYGON ((30.50671 50.41652, 30.49918 50.41632...","[(50.41652102209144, 30.506706589076238), (50....",1,4484.0,Київ,building,house,50.412965,30.510027
1,"POLYGON ((30.50671 50.41652, 30.49918 50.41632...","[(50.41652102209144, 30.506706589076238), (50....",1,243.0,Київ,amenity,school,50.414586,30.508348
1,"POLYGON ((30.50671 50.41652, 30.49918 50.41632...","[(50.41652102209144, 30.506706589076238), (50....",1,7430.0,Київ,building,apartments,50.412205,30.496817


In [22]:

def create_choropleth(data, json, columns, legend_name, feature, bins):
    lat, lon = get_lat_lon(data['geometry'])
    m = folium.Map(location=[sum(lat)/len(lat), sum(lon)/len(lon)], zoom_start=13, tiles='cartodbpositron')
    folium.Choropleth(
       geo_data=json,
        name="choropleth",
        data=data,
        columns=columns,
        key_on="feature.id",
        fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=legend_name,
        nan_fill_color = 'black',
        bins = bins
    ).add_to(m) 
    folium.LayerControl().add_to(m)

    return m

 

# подготовим данные

itog_table['geometry'] = itog_table['geometry'].astype(str) #для groupby

itog_table['id'] = itog_table['id'].astype(str) #для Choropleth

agg_all = itog_table.groupby(['geometry','type','id'], as_index = False).agg({'lat':'count'}).rename(columns = {'lat':'counts'})

agg_all['geometry'] = agg_all['geometry'].apply(wkt.loads) #возвращаем формат геометрий

 

agg_all_cafe = agg_all.query("type == 'cafe'")[["geometry","counts",'id']]

agg_all_cafe['id'] = agg_all_cafe['id'].astype(str)

data_geo_1 = gpd.GeoSeries(agg_all_cafe.set_index('id')["geometry"]).to_json()

 

create_choropleth(agg_all_cafe, data_geo_1, ["id","counts"], 'Cafe counts', 'counts', 5)

   

/usr/local/lib/python3.8/dist-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [23]:
 

gdf_aparts = ox.geometries_from_place(city, {'building' : 'apartments'}).reset_index()

# полнота building:levels (этажи)

print(np.round(len(gdf_aparts['building:levels'].dropna())/len(gdf_aparts['building:levels']), 2))

 

# полнота building:flats (квартиры)

#print(np.round(len(gdf_aparts['building:flats'].dropna())/len(gdf_aparts['building:flats']), 2))parts['building:flats'].dropna())/len(gdf_aparts['building:flats']), 2)

0.98


In [24]:
lat_g, lon_g = get_lat_lon(gdf_aparts['geometry'])
gdf_aparts['lat'] = lat_g
gdf_aparts['lon'] = lon_g

itog_table_people = itog_table.merge(gdf_aparts[['lat', 'lon', 'building:levels']], on = ['lat', 'lon'], how = 'left')
itog_table_people['building:levels'] = itog_table_people['building:levels'].fillna(1)
itog_table_people = itog_table_people.rename(columns = {'building:levels' : 'levels'})

apartments = ['apartments' , 'dormitory']
houses = ['house', 'semidetached_house', 'detached', 'terrace']
people_ctn = []

# в среднем возьмем 3 чел. на семью

for i in range(len(itog_table_people)):
    if itog_table_people['type'].iloc[i] in apartments:
        people = int(itog_table_people['levels'].iloc[i])*10*3
    elif itog_table_people['type'].iloc[i] in houses:
        people = int(itog_table_people['levels'].iloc[i])*3
    else:
        people = 'not living area'
    people_ctn.append(people)
itog_table_people['count_people'] = people_ctn
table_people = itog_table_people.query("count_people != 'not living area'")
table_people['count_people'] = table_people['count_people'].astype(int)

AttributeError: ignored